# code for stepik 

## 8.2 Transforming Distance Matrices into Evolutionary Trees

### step.12 Distances Between Leaves Problem

In [26]:
import numpy as np

def DFS_find_path(i,j,graph):
    stack = [(i, [i])]
    visited = set()
    while stack:
        (vertex, path) = stack.pop()
        if vertex not in visited:
            if vertex == j:
                return path
            visited.add(vertex)
            for neighbor in graph[vertex].keys():
                stack.append((neighbor, path + [neighbor]))

def AdjList2DisMatrix(n,adjList):
    disMatrix=np.zeros([n,n])
    graph={}
    for edge in adjList:
        start=int(edge.split(':')[0].split('->')[0])
        end=int(edge.split(':')[0].split('->')[1])
        weight=int(edge.split(':')[1])
        try:
            graph[start][end]=weight
        except:
            graph[start]={end:weight}
    print(graph)
    for i in range(n):
        for j in range(n):
            path=DFS_find_path(i,j,graph)
            score=0
            for k in range(len(path)-1):
                score+=graph[path[k]][path[k+1]]
            disMatrix[i][j]=score
            #print(i,j)
    print(disMatrix)
    return disMatrix
    
n=4
adjList='0->4:11 \
1->4:2 \
2->5:6 \
3->5:7 \
4->0:11 \
4->1:2 \
4->5:4 \
5->4:4 \
5->3:7 \
5->2:6'.split(' ')
disMatrix=AdjList2DisMatrix(n,adjList)

for y in disMatrix:
    print(' '.join([str(int(x)) for x in y]))

{0: {4: 11}, 1: {4: 2}, 2: {5: 6}, 3: {5: 7}, 4: {0: 11, 1: 2, 5: 4}, 5: {4: 4, 3: 7, 2: 6}}
[[ 0. 13. 21. 22.]
 [13.  0. 12. 13.]
 [21. 12.  0. 13.]
 [22. 13. 13.  0.]]
0 13 21 22
13 0 12 13
21 12 0 13
22 13 13 0


In [28]:
with open('ch8_data/dataset_369348_12.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    adjList=lines[1::]
    disMatrix=AdjList2DisMatrix(n,adjList)

for y in disMatrix:
    print(' '.join([str(int(x)) for x in y]))

{60: {61: 5, 12: 10, 59: 6}, 61: {60: 5, 23: 12, 56: 13}, 23: {61: 12}, 56: {61: 13, 2: 7, 55: 12}, 12: {60: 10}, 59: {60: 6, 27: 7, 58: 7}, 27: {59: 7}, 58: {59: 7, 14: 9, 57: 6}, 14: {58: 9}, 57: {58: 6, 8: 13, 42: 12}, 8: {57: 13}, 42: {57: 12, 17: 15, 41: 14}, 2: {56: 7}, 55: {56: 12, 5: 6, 54: 12}, 5: {55: 6}, 54: {55: 12, 24: 15, 53: 12}, 24: {54: 15}, 53: {54: 12, 9: 11, 52: 15}, 9: {53: 11}, 52: {53: 15, 20: 10, 51: 12}, 20: {52: 10}, 51: {52: 12, 21: 5, 50: 5}, 21: {51: 5}, 50: {51: 5, 22: 13, 49: 10}, 22: {50: 13}, 49: {50: 10, 30: 13, 48: 8}, 30: {49: 13}, 48: {49: 8, 7: 7, 47: 5}, 7: {48: 7}, 47: {48: 5, 28: 13, 46: 11}, 28: {47: 13}, 46: {47: 11, 15: 5, 45: 15}, 15: {46: 5}, 45: {46: 15, 6: 5, 44: 5}, 6: {45: 5}, 44: {45: 5, 19: 6, 43: 7}, 19: {44: 6}, 43: {44: 7, 10: 15, 18: 5}, 10: {43: 15}, 18: {43: 5}, 17: {42: 15}, 41: {42: 14, 3: 14, 40: 14}, 3: {41: 14}, 40: {41: 14, 0: 14, 39: 13}, 0: {40: 14}, 39: {40: 13, 16: 12, 38: 7}, 16: {39: 12}, 38: {39: 7, 25: 5, 37: 8}, 2

## 8.3 Toward An Algorithm for Distance-Based Phylogeny Construction

### Limb Length Problem

In [31]:
import numpy as np

def Limb_Length(j,n,disMatrix):
    min_limb=np.inf
    for i in range(n):
        for k in range(n):
            if i!=j and i!=k and j!=k:
                limb=(disMatrix[i][j]+disMatrix[j][k]-disMatrix[i][k])/2
                if limb<min_limb:
                    min_limb=limb
    return min_limb

n=4
j=1
disMatrix=[[0,13,21,22],
          [13,0,12,13],
          [21,12,0,13],
          [22,13,13,0]]
Limb_Length(j,n,disMatrix)

2.0

In [33]:
with open('ch8_data/dataset_369349_11.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    j=int(lines[1])
    disMatrix=[]
    for x in lines[2::]:
        disMatrix.append([float(y) for y in x.split(' ')])
Limb_Length(j,n,disMatrix)

373.0

## 8.4 Additive Phylogeny

### AdditivePhylogeny

In [9]:
import numpy as np

def Limb_Length(disMatrix,j):
    min_limb=np.inf
    n=len(disMatrix)
    for i in range(n):
        for k in range(n):
            if i!=j and i!=k and j!=k:
                limb=(disMatrix[i][j]+disMatrix[j][k]-disMatrix[i][k])/2
                if limb<min_limb:
                    min_limb=limb
    return int(min_limb)

def find_ik(disMatrix):
    n=len(disMatrix)
    for i in range(n-1):
        for k in range(n-1):
            if disMatrix[i][k]==disMatrix[i][n-1]+disMatrix[n-1][k] and i!=k:
                return i,k

def DFS_find_path_ik(edges,weights,i,k):
    stack=[[i]]
    visited=[i]
    findPath=[]
    while len(stack)>0:
        path=stack.pop()
        node=path[-1]
        visited.append(node)
        if node==k:
            findPath=path
            break
        for child in edges[node]:
            if child not in visited:
                stack.append(path+[child])
    return findPath

def find_v(edges,weights,x,findPath):
    total_distance=0
    for z in range(len(findPath)-1):
        close2i,close2k=findPath[z],findPath[z+1]
        # find v between i->k, where dist(i,v)=iv_distance,
        # i -> n1 -> n2 -> v -> n3 -> k, 
        # dist(i->n2) < x
        # dist(i->n2)+dist(n2->n3)>x, so v is between n2->n3
        if total_distance+weights[(close2i,close2k)]>x:
            return (close2i,close2k,x-total_distance,weights[(close2i,close2k)]+total_distance-x)
        total_distance+=weights[(close2i,close2k)]

def AdditivePhylogeny(disMatrix, n, nn):

    # here we maintain edges={} to store edges and weight={} to store weights
    if n==2:
        edges={0:[1],1:[0]}
        weights={(0,1):disMatrix[0][1],(1,0):disMatrix[1][0]}
        return edges,weights,nn

    limbLength = Limb_Length(disMatrix,n-1)
    for i in range(n):
        disMatrix[i][-1]-=limbLength
        disMatrix[-1][i]-=limbLength

    i,k=find_ik(disMatrix)
    x=disMatrix[i,-1]
    disMatrix_trimmed=disMatrix[:-1,:-1]
    edges,weights,nn=AdditivePhylogeny(disMatrix_trimmed, n-1, nn)
    findPath=DFS_find_path_ik(edges,weights,i,k)
    close2i,close2k,dis_i_close2i,dis_i_close2k=find_v(edges,weights,x,findPath)
    new_node=close2i

    if dis_i_close2i==0:
        pass
    else:
        new_node=nn
        nn+=1
        edges[close2i].remove(close2k)
        edges[close2k].remove(close2i)
        del weights[(close2i, close2k)]
        del weights[(close2k, close2i)]
        edges[close2i].append(new_node)
        edges[close2k].append(new_node)
        edges[new_node]=[close2i, close2k]
        weights[(new_node,close2i)]=dis_i_close2i
        weights[(close2i,new_node)]=dis_i_close2i
        weights[(new_node,close2k)]=dis_i_close2k
        weights[(close2k,new_node)]=dis_i_close2k
        
    # add leaf n back to T by creating a limb (v, n) of length Limb_Length
    edges[new_node].append(n-1)
    edges[n-1]=[new_node]
    weights[(n-1,new_node)]=limbLength
    weights[(new_node, n-1)]=limbLength
    return edges,weights,nn


def Format_output(edges,weights):
    for i in sorted(edges):
        for j in sorted(edges[i]):
            print (str(i)+"->"+str(j)+":"+str(weights[(i, j)]))

n=4
disMatrix=np.array([[0,13,21,22],
          [13,0,12,13],
          [21,12,0,13],
          [22,13,13,0]])
edges,weights,_= AdditivePhylogeny(disMatrix,n,n)
Format_output(edges,weights)

0->4:11
1->4:2
2->5:6
3->5:7
4->0:11
4->1:2
4->5:4
5->2:6
5->3:7
5->4:4


In [11]:
with open('ch8_data/dataset_369350_6.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    disMatrix=[]
    for x in lines[1::]:
        disMatrix.append([float(y) for y in x.split(' ')])
    disMatrix=np.array(disMatrix)
edges,weights,_= AdditivePhylogeny(disMatrix,n,n)
Format_output(edges,weights)

0->29:135.0
1->28:383.0
2->47:772.0
3->30:981.0
4->45:973.0
5->31:787
6->32:687
7->33:943
8->34:756
9->35:533
10->36:516
11->37:763
12->38:615
13->39:802
14->40:763
15->41:196
16->42:887
17->43:651
18->44:763
19->45:991
20->46:548
21->48:689.0
22->48:829
23->49:932
24->50:926
25->51:758
26->52:202
27->53:722
28->1:383.0
28->32:937.0
28->49:640.0
29->0:135.0
29->30:524.0
29->33:770.0
30->3:981.0
30->29:524.0
30->53:475.0
31->5:787
31->40:821.0
31->50:385.0
32->6:687
32->28:937.0
32->52:661.0
33->7:943
33->29:770.0
33->37:94.0
34->8:756
34->36:533.0
34->45:970.0
35->9:533
35->36:975.0
35->38:625.0
36->10:516
36->34:533.0
36->35:975.0
37->11:763
37->33:94.0
37->49:982.0
38->12:615
38->35:625.0
38->42:263.0
39->13:802
39->43:814.0
39->53:728.0
40->14:763
40->31:821.0
40->41:802.0
41->15:196
41->40:802.0
41->42:133.0
42->16:887
42->38:263.0
42->41:133.0
43->17:651
43->39:814.0
43->51:941.0
44->18:763
44->50:98.0
44->51:903.0
45->4:973.0
45->19:991
45->34:970.0
46->20:548
46->47:972.0
46->52

## 8.5 Using Least Squares to Construct Approximate Phylogenies

### Compute Discrepancy(﻿T, D).

In [12]:
import numpy as np

def Discrepancy(T, D):
    out=0
    for i in range(len(T)):
        for j in range(i,len(T[0])): # important, take only half
            out+=np.square(T[i][j]-D[i][j])
            print(T[i][j],D[i][j])
    return out

T=[[0,7,9,10],
  [7,0,10,11],
  [9,10,0,3],
  [10,11,3,0]]

D=[[0,3,4,3],
  [3,0,4,5],
  [4,4,0,2],
  [3,5,2,0]]

Discrepancy(T, D)

0 0
7 3
9 4
10 3
0 0
10 4
11 5
0 0
3 2
0 0


163

## 8.6 Ultrametric Evolutionary Trees

### UPGMA

In [76]:
class Tree(object):
    
    ## public code for the data structure Tree
    
    def __init__(self,N=-1,bidirectional=True):
        self.nodes=list(range(N))
        self.edges={}
        self.bidirectional=bidirectional
        self.N = N
        
    def link(self,start,end,weight=1): 
        self.half_link(start,end,weight)
        if self.bidirectional:
            self.half_link(end,start,weight)
        
    def unlink(self,i,k):
        try:
            self.half_unlink(i,k)
            if self.bidirectional:
                self.half_unlink(k,i)
        except KeyError:
            print ('Could not unlink {0} from {1}'.format(i,k))
            self.print()        
        
    def half_link(self,a,b,weight=1):
        if not a in self.nodes:
            self.nodes.append(a)        
        if a in self.edges:               
            self.edges[a]=[(b0,w0) for (b0,w0) in self.edges[a] if b0 != b] + [(b,weight)]
        else:
            self.edges[a]=[(b,weight)]
    
    def half_unlink(self,a,b):
        links=[(e,w) for (e,w) in self.edges[a] if e != b]
        if len(links)<len(self.edges[a]):
            self.edges[a]=links
        else:
            print ('Could not unlink {0} from {1}'.format(a,b))
            self.print()
    
    def are_linked(self,a,b):
        return len([e for (e,w) in self.edges[a] if e == b])>0
        
    def AdjList(self,includeNodes=False):
        print('AdjList')
        self.nodes.sort()
        if includeNodes:
            print (self.nodes)
        for node in self.nodes:
            if node in self.edges:
                for edge in self.edges[node]:
                    end,weight=edge
                    print ('%i->%i:%.3f'%(node,end,weight))
                    
    def num_nodes(self):
        return len(self.nodes)
    
    def traverse(self,i,k,path=[],weights=[]):
        if not i in self.edges: return (False,[])
        if len(path)==0:
            path=[i]
            weights=[0]
        
        for j,w in self.edges[i]:
            if j in path: continue
            path1=path + [j]
            weights1=weights+[w]
            if j==k:
                return (True,list(zip(path1,weights1)))
            else:
                found_k,test=self.traverse(j,k,path1,weights1)
                if found_k: 
                    return (found_k,test)
        return (False,[])  
    
    def get_nodes(self):
        for node in self.nodes:
            yield(node)
            
    def initialize_from(self,T):
        for node in T.nodes:
            if not node in self.nodes:
                self.nodes.append(node)
                if node in T.edges:
                    for a,w in T.edges[node]:
                        self.link(node,a,w)
     
    def get_links(self):
        return [(a,b,w) for a in self.nodes for (b,w) in self.edges[a] if a in self.edges]
    
    def remove_backward_links(self,root):
        self.bidirectional = False
        for (child,_) in self.edges[root]:
            self.half_unlink(child,root)
            self.remove_backward_links(child)

import numpy as np

def find2ClosestClusters(disMatrix,Clusters):
    smallest_i=-1
    smallest_j=-1
    smallest_dis=np.inf
    for i in range(len(disMatrix)):
        for j in range(i):
            if i in Clusters and j in Clusters and disMatrix[i][j]<smallest_dis:
                smallest_i=i
                smallest_j=j
                smallest_dis=disMatrix[i][j]
    return (smallest_i,smallest_j,smallest_dis)  
    
def D(i,j,disMatrix,Clusters):
    if (i in Clusters)&(j in Clusters):
        tmp_sum=0
        for ii in Clusters[i]:
            for jj in Clusters[j]:
                tmp_sum+=disMatrix[ii][jj]
        return tmp_sum/(len(Clusters[i])*len(Clusters[j]))
    else:
        return None
    
def UPGMA(disMatrix,n):
    global T
    Clusters={}
    for i in range(n):
        Clusters[i]=[i]
    T=Tree(n)
    Age={}
    for v in T.get_nodes():
        Age[v]=0
    while len(Clusters)>1:
        Ci,Cj,distance=find2ClosestClusters(disMatrix,Clusters)
        Cnew=T.num_nodes()
        T.link(Cnew,Ci)
        T.link(Cnew,Cj)        
        Clusters[Cnew]=Clusters[Ci]+Clusters[Cj]
        Age[Cnew]=disMatrix[Ci][Cj]/2
        del Clusters[Ci],Clusters[Cj]
        Cnew_Data=[D(i,Cnew,disMatrix,Clusters) for i in range(len(disMatrix))] # add the right column
        for row in range(len(disMatrix)):
            disMatrix[row].append(Cnew_Data[row])
        disMatrix.append(Cnew_Data+[0]) # add the new row at the bottom
    for node in T.nodes:
        new_data=[]
        for child,weight in T.edges[node]:
            new_data.append((child,np.absolute(Age[node]-Age[child])))
        T.edges[node]=new_data
    return T

n=4
disMatrix=[[0,20,17,11],
          [20,0,20,13],
          [17,20,0,10],
          [11,13,10,0]]

UPGMA(disMatrix,n).AdjList()

AdjList
0->5:7.000
1->6:8.833
2->4:5.000
3->4:5.000
4->3:5.000
4->2:5.000
4->5:2.000
5->4:2.000
5->0:7.000
5->6:1.833
6->5:1.833
6->1:8.833


In [81]:
with open('ch8_data/dataset_369352_8.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    disMatrix=[]
    for x in lines[1::]:
        disMatrix.append([float(y) for y in x.split(' ')[:-1]])
UPGMA(disMatrix,n).AdjList()

AdjList
0->26:268.000
1->29:277.500
2->35:343.500
3->31:282.500
4->24:265.500
5->33:305.500
6->24:265.500
7->23:265.000
8->23:265.000
9->31:282.500
10->27:269.500
11->32:295.500
12->25:266.000
13->28:271.000
14->28:271.000
15->34:326.000
16->26:268.000
17->30:281.250
18->29:277.500
19->32:295.500
20->34:326.000
21->27:269.500
22->25:266.000
23->8:265.000
23->7:265.000
23->37:87.875
24->6:265.500
24->4:265.500
24->33:40.000
25->22:266.000
25->12:266.000
25->37:86.875
26->16:268.000
26->0:268.000
26->35:75.500
27->21:269.500
27->10:269.500
27->36:76.750
28->14:271.000
28->13:271.000
28->30:10.250
29->18:277.500
29->1:277.500
29->38:77.500
30->28:10.250
30->17:281.250
30->41:110.250
31->9:282.500
31->3:282.500
31->36:63.750
32->19:295.500
32->11:295.500
32->38:59.500
33->24:40.000
33->5:305.500
33->40:84.958
34->20:326.000
34->15:326.000
34->42:74.688
35->26:75.500
35->2:343.500
35->41:48.000
36->31:63.750
36->27:76.750
36->39:40.062
37->25:86.875
37->23:87.875
37->39:33.438
38->32:59.500

## 8.7 The Neighbor-Joining Algorithm

### TotalDistance, D*

In [86]:
import numpy as np

def TotalDistance(disMatrix):
    totalDistance=[]
    for x in disMatrix:
        totalDistance.append(np.sum(x))
    return totalDistance

def D_(disMatrix):
    n=len(disMatrix)
    d_=np.zeros([n,n])
    totalDistance=TotalDistance(disMatrix)
    for i in range(n):
        for j in range(n):
            if i!=j:
                d_[i][j]=(n-2)*disMatrix[i][j]-totalDistance[i]-totalDistance[j]
    return d_
    

disMatrix=[[0,13,21,22],
          [13,0,12,13],
          [21,12,0,13],
          [22,13,13,0]]
print(TotalDistance(disMatrix))
print(D_(disMatrix))

[56, 38, 46, 48]
[[  0. -68. -60. -60.]
 [-68.   0. -60. -60.]
 [-60. -60.   0. -68.]
 [-60. -60. -68.   0.]]


### NeighborJoining

In [17]:
class Tree(object):
    
    ## public code for the data structure Tree
    
    def __init__(self,N=-1,bidirectional=True):
        self.nodes=list(range(N))
        self.edges={}
        self.bidirectional=bidirectional
        self.N = N
        
    def link(self,start,end,weight=1): 
        self.half_link(start,end,weight)
        if self.bidirectional:
            self.half_link(end,start,weight)
        
    def unlink(self,i,k):
        try:
            self.half_unlink(i,k)
            if self.bidirectional:
                self.half_unlink(k,i)
        except KeyError:
            print ('Could not unlink {0} from {1}'.format(i,k))
            self.print()        
        
    def half_link(self,a,b,weight=1):
        if not a in self.nodes:
            self.nodes.append(a)        
        if a in self.edges:               
            self.edges[a]=[(b0,w0) for (b0,w0) in self.edges[a] if b0 != b] + [(b,weight)]
        else:
            self.edges[a]=[(b,weight)]
    
    def half_unlink(self,a,b):
        links=[(e,w) for (e,w) in self.edges[a] if e != b]
        if len(links)<len(self.edges[a]):
            self.edges[a]=links
        else:
            print ('Could not unlink {0} from {1}'.format(a,b))
            self.print()
    
    def are_linked(self,a,b):
        return len([e for (e,w) in self.edges[a] if e == b])>0
        
    def AdjList(self,includeNodes=False):
        print('AdjList')
        self.nodes.sort()
        if includeNodes:
            print (self.nodes)
        for node in self.nodes:
            if node in self.edges:
                for edge in self.edges[node]:
                    end,weight=edge
                    print ('%i->%i:%.3f'%(node,end,weight))
                    
    def num_nodes(self):
        return len(self.nodes)
    
    def traverse(self,i,k,path=[],weights=[]):
        if not i in self.edges: return (False,[])
        if len(path)==0:
            path=[i]
            weights=[0]
        
        for j,w in self.edges[i]:
            if j in path: continue
            path1=path + [j]
            weights1=weights+[w]
            if j==k:
                return (True,list(zip(path1,weights1)))
            else:
                found_k,test=self.traverse(j,k,path1,weights1)
                if found_k: 
                    return (found_k,test)
        return (False,[])  
    
    def get_nodes(self):
        for node in self.nodes:
            yield(node)
            
    def initialize_from(self,T):
        for node in T.nodes:
            if not node in self.nodes:
                self.nodes.append(node)
                if node in T.edges:
                    for a,w in T.edges[node]:
                        self.link(node,a,w)
     
    def get_links(self):
        return [(a,b,w) for a in self.nodes for (b,w) in self.edges[a] if a in self.edges]
    
    def remove_backward_links(self,root):
        self.bidirectional = False
        for (child,_) in self.edges[root]:
            self.half_unlink(child,root)
            self.remove_backward_links(child)

import numpy as np

def TotalDistance(disMatrix):
    totalDistance=[]
    for x in disMatrix:
        totalDistance.append(np.sum(x))
    return totalDistance

def D_(disMatrix):
    n=len(disMatrix)
    d_=np.zeros([n,n])
    totalDistance=TotalDistance(disMatrix)
    for i in range(n):
        for j in range(n):
            if i!=j:
                d_[i][j]=(n-2)*disMatrix[i][j]-totalDistance[i]-totalDistance[j]
    return d_

def findijFromDStar(D_Star):
    smallest=np.inf
    smallest_i=-1
    smallest_j=-1
    for i in range(len(D_Star)):
        for j in range(i,len(D_Star)):
            if D_Star[i][j]<smallest and i!=j:
                smallest=D_Star[i][j]
                smallest_i=i
                smallest_j=j
    return smallest_i,smallest_j

def remove_ij_row_column(DD,ii,jj):
    new_D=[]
    for i in range(len(DD)):
        tmp=[]
        for j in range(len(DD[0])):
            if j!=ii and j!=jj:
                tmp.append(DD[i][j])
        if i!=ii and i!=jj:
            new_D.append(tmp)
    return new_D
    
# modified from AdditivePhylogeny

def NeighborJoining(D,Nodes=None):
    # ColumnHeader makes the node in D, eg: [2,3,6]
    n=len(D)
    #print(D)
    if Nodes==None:
        Nodes=[x for x in range(n)]
    if n==2:
        T=Tree()
        T.link(Nodes[0],Nodes[1],D[0][1])
        return T
    totalDistance=TotalDistance(D)
    D_star=D_(D)
    i,j=findijFromDStar(D_star)
    delta=(totalDistance[i]-totalDistance[j])/(n-2)
    limbLengthi=(D[i][j]+delta)/2
    limbLengthj=(D[i][j]-delta)/2
    #print(i,j,n,D)
    new_Data=[(D[k][i]+D[k][j]-D[i][j])/2 for k in range(n)]
    for row in range(n):
        D[row].append(new_Data[row])
    D.append(new_Data+[0])
    #print(Nodes)
    m=Nodes[-1]+1
    Nodes.append(m)
    #print(D)
    D=remove_ij_row_column(D,i,j) # D with rows/column i and j removed
    #print(D)
    i=Nodes[i]
    j=Nodes[j]
    #print(Nodes,node_i,node_j)
    Nodes.remove(i)
    Nodes.remove(j)
    T=NeighborJoining(D,Nodes)
    T.link(i,m,limbLengthi)
    T.link(j,m,limbLengthj)
    return T

n=4
disMatrix=[[0,23,27,20],
          [23,0,30,28],
          [27,30,0,30],
          [20,28,30,0]]

NeighborJoining(disMatrix).AdjList()

AdjList
0->4:8.000
1->5:13.500
2->5:16.500
3->4:12.000
4->5:2.000
4->0:8.000
4->3:12.000
5->4:2.000
5->1:13.500
5->2:16.500


In [18]:
with open('ch8_data/dataset_369353_7.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    disMatrix=[]
    for x in lines[1::]:
        disMatrix.append([float(y) for y in x.split(' ')[:-1]])
NeighborJoining(disMatrix).AdjList()

AdjList
0->36:513.740
1->48:470.301
2->34:504.973
3->32:483.717
4->49:551.644
5->38:528.521
6->37:559.770
7->45:524.647
8->44:516.649
9->33:509.784
10->35:537.009
11->41:493.327
12->44:540.351
13->36:521.260
14->39:570.207
15->40:518.642
16->45:519.353
17->35:521.991
18->34:536.027
19->32:576.283
20->58:656.802
21->49:525.356
22->39:484.793
23->51:582.920
24->33:536.216
25->38:503.479
26->37:479.230
27->42:560.416
28->51:480.080
29->43:538.464
30->50:569.641
31->47:478.123
32->48:147.199
32->3:483.717
32->19:576.283
33->57:205.650
33->9:509.784
33->24:536.216
34->43:77.536
34->2:504.973
34->18:536.027
35->41:54.173
35->10:537.009
35->17:521.991
36->46:82.572
36->0:513.740
36->13:521.260
37->46:80.928
37->6:559.770
37->26:479.230
38->40:37.358
38->5:528.521
38->25:503.479
39->53:164.667
39->14:570.207
39->22:484.793
40->42:21.084
40->15:518.642
40->38:37.358
41->52:149.634
41->11:493.327
41->35:54.173
42->53:154.521
42->27:560.416
42->40:21.084
43->55:161.149
43->29:538.464
43->34:77.53

## 8.9 The Small Parsimony Problem

### SmallParsimony

In [1]:
# public code for Tree structure

import re,random

class Tree(object):
    
    ## public code for the data structure Tree
    
    def __init__(self,N=-1,bidirectional=True):
        self.nodes=list(range(N))
        self.edges={}
        self.bidirectional=bidirectional
        self.N = N
        
    def link(self,start,end,weight=1): 
        self.half_link(start,end,weight)
        if self.bidirectional:
            self.half_link(end,start,weight)
        
    def unlink(self,i,k):
        try:
            self.half_unlink(i,k)
            if self.bidirectional:
                self.half_unlink(k,i)
        except KeyError:
            print ('Could not unlink {0} from {1}'.format(i,k))
            self.print()        
        
    def half_link(self,a,b,weight=1):
        if not a in self.nodes:
            self.nodes.append(a)        
        if a in self.edges:               
            self.edges[a]=[(b0,w0) for (b0,w0) in self.edges[a] if b0 != b] + [(b,weight)]
        else:
            self.edges[a]=[(b,weight)]
    
    def half_unlink(self,a,b):
        links=[(e,w) for (e,w) in self.edges[a] if e != b]
        if len(links)<len(self.edges[a]):
            self.edges[a]=links
        else:
            print ('Could not unlink {0} from {1}'.format(a,b))
            self.print()
    
    def are_linked(self,a,b):
        return len([e for (e,w) in self.edges[a] if e == b])>0
        
    def AdjList(self,includeNodes=False):
        print('AdjList')
        self.nodes.sort()
        if includeNodes:
            print (self.nodes)
        for node in self.nodes:
            if node in self.edges:
                for edge in self.edges[node]:
                    end,weight=edge
                    print ('%i->%i:%.3f'%(node,end,weight))
                    
    def num_nodes(self):
        return len(self.nodes)
    
    def traverse(self,i,k,path=[],weights=[]):
        if not i in self.edges: return (False,[])
        if len(path)==0:
            path=[i]
            weights=[0]
        
        for j,w in self.edges[i]:
            if j in path: continue
            path1=path + [j]
            weights1=weights+[w]
            if j==k:
                return (True,list(zip(path1,weights1)))
            else:
                found_k,test=self.traverse(j,k,path1,weights1)
                if found_k: 
                    return (found_k,test)
        return (False,[])  
    
    def get_nodes(self):
        for node in self.nodes:
            yield(node)
     
    def get_links(self):
        return [(a,b,w) for a in self.nodes for (b,w) in self.edges[a] if a in self.edges]
    
    def remove_backward_links(self,root):
        self.bidirectional = False
        for (child,_) in self.edges[root]:
            self.half_unlink(child,root)
            self.remove_backward_links(child)
            
class NewTree(Tree):
        
    @staticmethod
    def buildFromAdjList(lines,letters='ATGC',bidirectional=False):
        def get_leaf(string):
            for index,label in T.labels.items():
                if string==label:
                    return index
            return None
        
        T=NewTree(bidirectional=bidirectional)
        pattern=re.compile('(([0-9]+)|([{0}]+))->(([0-9]+)|([{0}]+))'.format(letters))
        
        for line in lines:
            m=pattern.match(line)
            if m:
                i=None
 
                if m.group(2)==None:
                    i=get_leaf(m.group(3))
                    if i==None:
                        i=len(T.labels)
                    T.set_label(i,m.group(3))
 
                    if not i in T.leaves:
                        T.leaves.append(i)
                        
                if m.group(3)==None:
                    i=int(m.group(2))
           
                if m.group(5)==None:
                    j=get_leaf(m.group(6))
                    if j==None:
                        j=len(T.labels)
                        if not j in T.leaves:
                            T.leaves.append(j)                        
                        T.set_label(j,m.group(6))
                    T.link(i,j)
                   
                if m.group(6)==None:
                    j=int(m.group(5))
                    T.link(i,j)
                    
        return T
    
    def __init__(self,N=-1,bidirectional=True):
        super().__init__(N,bidirectional=bidirectional)
        self.labels={}
        self.leaves=[]
        
    def is_leaf(self,v):
        return v in self.leaves
    
    def set_label(self,node,label):
        if not node in self.nodes:
            self.nodes.append(node)
        self.labels[node]=label
            
def HamDistance(s,t):
    return len([a for (a,b) in zip(s,t) if a!=b])

In [2]:
import numpy as np

def findRipe(T,visited):
    for v in T.nodes:
        if not visited[v] and v in T.edges:
            for i,_ in T.edges[v]:
                if i>v:
                    continue
                if visited[i]==False:
                    break
            return v # all chlilds of v are visited, but v is not visited (Tag(v)=0)
    return None


def CalScore(alp,v,scores,T,alphabet='ATGC'):
    tmp=[]
    for e,_ in T.edges[v]:
        ttmp=[]
        for i in range(len(alphabet)):
            if alp==alphabet[i]:
                delta=0
            else:
                delta=1
            ttmp.append(scores[e][i]+delta)
        tmp.append(np.min(ttmp))
    return sum(tmp)
        
def UpdateOutTree(v,scores,OutTree,alphabet='ATGC'):
    if v not in OutTree.labels:
        OutTree.labels[v]=''
    ii=0
    smallest_score=np.inf
    for i in range(len(scores)):
        if scores[i]<smallest_score:
            smallest_score=scores[i]
            ii=i
    OutTree.set_label(v,OutTree.labels[v]+alphabet[ii])
    return alphabet[ii]
        
def Backtrack(v,character_assign,scores,T,OutTree,alphabet='ATGC'):
    for v_,_ in T.edges[v]:
        if T.is_leaf(v_):
            continue
        if v_ not in OutTree.labels:
            OutTree.labels[v_]=''
        smallest_score=np.inf
        for i in range(len(alphabet)):
            if scores[v_][i]<smallest_score:
                smallest_score=scores[v_][i]
        moves=[i for i in range(len(alphabet)) if scores[v_][i]==smallest_score]
        flag=False
        for i in moves:
            if alphabet[i]==character_assign:        
                flag=True
                OutTree.set_label(v_,OutTree.labels[v_]+character_assign)
                Backtrack(v_,character_assign,scores,T,OutTree) 
        if flag==False:
            random_assign=alphabet[moves[random.randrange(0,(len(moves)))]]               
            OutTree.set_label(v_,OutTree.labels[v_]+random_assign)
            Backtrack(v_,random_assign,scores,T,OutTree)   

def SmallParsimonyCharacter(Character,OutTree,T,alphabet='ATGC'):
            
    # new tree for solving single character smallest score
    
    visited={}
    scores={}
    for v in T.nodes:
        if T.is_leaf(v):
            visited[v]=True
            scores[v]=[]
            for alp in alphabet:
                if Character[v]==alp:
                    scores[v].append(0)
                else:
                    scores[v].append(np.inf)
        else:
            visited[v]=False
    v=findRipe(T,visited)
    while v!=None:
        visited[v]=True
        scores[v]=[CalScore(alp,v,scores,T) for alp in alphabet]
        _v=v # keep the last v
        v=findRipe(T,visited) # update all Ripe in T
    character_assign=UpdateOutTree(_v,scores[_v],OutTree)
    Backtrack(_v,character_assign,scores,T,OutTree)
    tmp=[]
    for i in range(len(alphabet)):
        tmp.append(scores[_v][i])
    smallest_score=min(tmp)
    return smallest_score
        
def SmallParsimony(T):
 
    # use recursion to get the minimal value
    OutTree=NewTree(T.N)
    
    for node in T.nodes:
        if not node in OutTree.nodes:
            OutTree.nodes.append(node)
            if node in T.edges:
                for a,w in T.edges[node]:
                    OutTree.link(node,a,w)
    for node,label in T.labels.items():
        OutTree.set_label(node,label)
        
    tmp=[]
    #print(T.labels)
    for i in range(len(T.labels[0])): # for each position
        tmp.append(SmallParsimonyCharacter([label[i] for label in T.labels.values()],OutTree,T))
    scores=sum(tmp)
    
    return scores,OutTree

def Format_OutTree(OutTree):
    AllNodes=sorted(OutTree.nodes)
    AllEdges=OutTree.edges
    AllLabels=OutTree.labels
    #print(AllNodes,AllEdges,AllLabels)
    for v in AllNodes:
        if v in AllEdges:
            for w,weight in AllEdges[v]:
                if v in AllLabels and w in AllLabels:
                    print ('{0}->{1}:{2}'.format(AllLabels[v],AllLabels[w],HamDistance(AllLabels[v],AllLabels[w])))   

n=4
AdjList=['4->CAAATCCC','4->ATTGCGAC','5->CTGCGCTG','5->ATGGACGA','6->4','6->5']
T=NewTree.buildFromAdjList(AdjList)
score,OutTree=SmallParsimony(T)
print(score)
Format_OutTree(OutTree)

16.0
CAAATCCC->ATAGTCAC:4
ATTGCGAC->ATAGTCAC:3
CTGCGCTG->ATGGACGA:5
ATGGACGA->ATGGACGA:0
ATAGTCAC->CAAATCCC:4
ATAGTCAC->ATTGCGAC:3
ATAGTCAC->ATAGACAA:2
ATGGACGA->CTGCGCTG:5
ATGGACGA->ATGGACGA:0
ATGGACGA->ATAGACAA:2
ATAGACAA->ATAGTCAC:2
ATAGACAA->ATGGACGA:2


In [3]:
with open('ch8_data/dataset_369355_10.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    AdjList=lines[1::]
T=NewTree.buildFromAdjList(AdjList)
score,OutTree=SmallParsimony(T)
print(score)
Format_OutTree(OutTree)

11857.0
TATCTCAATGCATCCCGAACCTTCGCACTAGCCAAGGTCTGGGTATATTACTTGTTCTTGAACTGGCGTCCGAATGCATACCACGCCGTCCAGGGCTGAGGTCGGTCAGGGCTGGTTTCTGATTGCCTAGGACTCGTTATGTCCAGTCGCATGGCTATACAGTCACCCA->TATTTCAAGGCTTCTCCCCCCTTAATGCTACCTATCACCCGGGTATTGATCAAGTTCCTGGCTGAGATTCCGATTTCATACCGCGCCATACCGGTGAGAGGTTGGCCAGGCCTGGTCGCCGATCGTCCATGACTCGTTAATTTCAGCCTCGTTGCTATCCGGTAATCTG:64
ATTTTGCGGTCTACTCCCCTATCAATGCAACCTATCACACCCCCACTGATAAAGTAACTAGCTGATATAGCGGTGTCGCTCTGGGACAAACCGCTGACAGATTGTCCGGACTGTTTCGTCAATCTTACGTAATCGCTCGATCTGACCCTTGCTGCTATCGGGGAATTTG->TATTTCAAGGCTTCTCCCCCCTTAATGCTACCTATCACCCGGGTATTGATCAAGTTCCTGGCTGAGATTCCGATTTCATACCGCGCCATACCGGTGAGAGGTTGGCCAGGCCTGGTCGCCGATCGTCCATGACTCGTTAATTTCAGCCTCGTTGCTATCCGGTAATCTG:63
ATACTAGAGATAATGCACGCTTCCAAACCTGCGCGTAGTACGGTCCTCATGATCCGTTCACGGGAGCAAAAGATGGATCTGAACATAGAGGTGTTCTTGACACGATATCCCCCTTTTACTTTCCCAGCCGCAGACAAAAATATACCTCGCAAGTTGGTGTAGCTATATG->ATATTAAGGATAATGCCCGCCTGGCACCAGGCCCGCAGTATTGACCGCCCGATCCGCGAAACGGTCAATGAGATGCATCAGCACATAGGGATATCCAGGTCAAGATACACTCCTTCGAATTTTCTACTTGCAA

### Small Parsimony in an Unrooted Tree Problem

In [6]:
# public code for Tree structure

import re,random

class Tree(object):
    
    ## public code for the data structure Tree
    
    def __init__(self,N=-1,bidirectional=True):
        self.nodes=list(range(N))
        self.edges={}
        self.bidirectional=bidirectional
        self.N = N
        
    def link(self,start,end,weight=1): 
        self.half_link(start,end,weight)
        if self.bidirectional:
            self.half_link(end,start,weight)
        
    def unlink(self,i,k):
        try:
            self.half_unlink(i,k)
            if self.bidirectional:
                self.half_unlink(k,i)
        except KeyError:
            print ('Could not unlink {0} from {1}'.format(i,k))
            self.print()        
        
    def half_link(self,a,b,weight=1):
        if not a in self.nodes:
            self.nodes.append(a)        
        if a in self.edges:               
            self.edges[a]=[(b0,w0) for (b0,w0) in self.edges[a] if b0 != b] + [(b,weight)]
        else:
            self.edges[a]=[(b,weight)]
    
    def half_unlink(self,a,b):
        links=[(e,w) for (e,w) in self.edges[a] if e != b]
        if len(links)<len(self.edges[a]):
            self.edges[a]=links
        else:
            print ('Could not unlink {0} from {1}'.format(a,b))
            self.print()
    
    def are_linked(self,a,b):
        return len([e for (e,w) in self.edges[a] if e == b])>0
        
    def AdjList(self,includeNodes=False):
        print('AdjList')
        self.nodes.sort()
        if includeNodes:
            print (self.nodes)
        for node in self.nodes:
            if node in self.edges:
                for edge in self.edges[node]:
                    end,weight=edge
                    print ('%i->%i:%.3f'%(node,end,weight))
                    
    def num_nodes(self):
        return len(self.nodes)
    
    def traverse(self,i,k,path=[],weights=[]):
        if not i in self.edges: return (False,[])
        if len(path)==0:
            path=[i]
            weights=[0]
        
        for j,w in self.edges[i]:
            if j in path: continue
            path1=path + [j]
            weights1=weights+[w]
            if j==k:
                return (True,list(zip(path1,weights1)))
            else:
                found_k,test=self.traverse(j,k,path1,weights1)
                if found_k: 
                    return (found_k,test)
        return (False,[])  
    
    def get_nodes(self):
        for node in self.nodes:
            yield(node)
     
    def get_links(self):
        return [(a,b,w) for a in self.nodes for (b,w) in self.edges[a] if a in self.edges]
    
    def remove_backward_links(self,root):
        self.bidirectional = False
        for (child,_) in self.edges[root]:
            self.half_unlink(child,root)
            self.remove_backward_links(child)
            
class NewTree(Tree):
        
    @staticmethod
    def buildFromAdjList(lines,letters='ATGC',bidirectional=False):
        def get_leaf(string):
            for index,label in T.labels.items():
                if string==label:
                    return index
            return None
        
        T=NewTree(bidirectional=bidirectional)
        pattern=re.compile('(([0-9]+)|([{0}]+))->(([0-9]+)|([{0}]+))'.format(letters))
        
        for line in lines:
            m=pattern.match(line)
            if m:
                i=None
 
                if m.group(2)==None:
                    i=get_leaf(m.group(3))
                    if i==None:
                        i=len(T.labels)
                    T.set_label(i,m.group(3))
 
                    if not i in T.leaves:
                        T.leaves.append(i)
                        
                if m.group(3)==None:
                    i=int(m.group(2))
           
                if m.group(5)==None:
                    j=get_leaf(m.group(6))
                    if j==None:
                        j=len(T.labels)
                        if not j in T.leaves:
                            T.leaves.append(j)                        
                        T.set_label(j,m.group(6))
                    T.link(i,j)
                   
                if m.group(6)==None:
                    j=int(m.group(5))
                    T.link(i,j)
                    
        return T
    
    def __init__(self,N=-1,bidirectional=True):
        super().__init__(N,bidirectional=bidirectional)
        self.labels={}
        self.leaves=[]
        
    def is_leaf(self,v):
        return v in self.leaves
    
    def set_label(self,node,label):
        if not node in self.nodes:
            self.nodes.append(node)
        self.labels[node]=label
            
def HamDistance(s,t):
    return len([a for (a,b) in zip(s,t) if a!=b])

import numpy as np

def findRipe(T,visited):
    for v in T.nodes:
        if not visited[v] and v in T.edges:
            for i,_ in T.edges[v]:
                if i>v:
                    continue
                if visited[i]==False:
                    break
            return v # all chlilds of v are visited, but v is not visited (Tag(v)=0)
    return None


def CalScore(alp,v,scores,T,alphabet='ATGC'):
    tmp=[]
    for e,_ in T.edges[v]:
        ttmp=[]
        for i in range(len(alphabet)):
            if alp==alphabet[i]:
                delta=0
            else:
                delta=1
            ttmp.append(scores[e][i]+delta)
        tmp.append(np.min(ttmp))
    return sum(tmp)
        
def UpdateOutTree(v,scores,OutTree,alphabet='ATGC'):
    if v not in OutTree.labels:
        OutTree.labels[v]=''
    ii=0
    smallest_score=np.inf
    for i in range(len(scores)):
        if scores[i]<smallest_score:
            smallest_score=scores[i]
            ii=i
    OutTree.set_label(v,OutTree.labels[v]+alphabet[ii])
    return alphabet[ii]
        
def Backtrack(v,character_assign,scores,T,OutTree,alphabet='ATGC'):
    for v_,_ in T.edges[v]:
        if T.is_leaf(v_):
            continue
        if v_ not in OutTree.labels:
            OutTree.labels[v_]=''
        smallest_score=np.inf
        for i in range(len(alphabet)):
            if scores[v_][i]<smallest_score:
                smallest_score=scores[v_][i]
        moves=[i for i in range(len(alphabet)) if scores[v_][i]==smallest_score]
        flag=False
        for i in moves:
            if alphabet[i]==character_assign:        
                flag=True
                OutTree.set_label(v_,OutTree.labels[v_]+character_assign)
                Backtrack(v_,character_assign,scores,T,OutTree) 
        if flag==False:
            random_assign=alphabet[moves[random.randrange(0,(len(moves)))]]               
            OutTree.set_label(v_,OutTree.labels[v_]+random_assign)
            Backtrack(v_,random_assign,scores,T,OutTree)   

def SmallParsimonyCharacter(Character,OutTree,T,alphabet='ATGC'):
            
    # new tree for solving single character smallest score
    
    visited={}
    scores={}
    for v in T.nodes:
        if T.is_leaf(v):
            visited[v]=True
            scores[v]=[]
            for alp in alphabet:
                if Character[v]==alp:
                    scores[v].append(0)
                else:
                    scores[v].append(np.inf)
        else:
            visited[v]=False
    v=findRipe(T,visited)
    while v!=None:
        visited[v]=True
        scores[v]=[CalScore(alp,v,scores,T) for alp in alphabet]
        _v=v # keep the last v
        v=findRipe(T,visited) # update all Ripe in T
    character_assign=UpdateOutTree(_v,scores[_v],OutTree)
    Backtrack(_v,character_assign,scores,T,OutTree)
    tmp=[]
    for i in range(len(alphabet)):
        tmp.append(scores[_v][i])
    smallest_score=min(tmp)
    return smallest_score
        
def SmallParsimony(T):
 
    # use recursion to get the minimal value
    OutTree=NewTree(T.N)
    
    for node in T.nodes:
        if not node in OutTree.nodes:
            OutTree.nodes.append(node)
            if node in T.edges:
                for a,w in T.edges[node]:
                    OutTree.link(node,a,w)
    for node,label in T.labels.items():
        OutTree.set_label(node,label)
        
    tmp=[]
    #print(T.labels)
    for i in range(len(T.labels[0])): # for each position
        tmp.append(SmallParsimonyCharacter([label[i] for label in T.labels.values()],OutTree,T))
    scores=sum(tmp)
    
    return scores,OutTree

def Format_OutTree(OutTree):
    AllNodes=sorted(OutTree.nodes)
    AllEdges=OutTree.edges
    AllLabels=OutTree.labels
    #print(AllNodes,AllEdges,AllLabels)
    for v in AllNodes:
        if v in AllEdges:
            for w,weight in AllEdges[v]:
                if v in AllLabels and w in AllLabels:
                    print ('{0}->{1}:{2}'.format(AllLabels[v],AllLabels[w],HamDistance(AllLabels[v],AllLabels[w])))   

def SmallParsimony_Unrooted(T):
    
    # in this case, we can assign a root
    #print(T.nodes,T.edges,T.labels)
    node1=T.nodes[-1]
    node2=T.edges[node1][0][0]
    #print(node1,node2)
    T.unlink(node1,node2)
    node3=len(T.nodes) # then node3 is the new root
    T.link(node3,node1)
    T.link(node3,node2)
    print(node1,node2,node3)
    #"""
    T.remove_backward_links(node3)
    scores,OutTree=SmallParsimony(T)
    OutTree.unlink(node3,node1)
    OutTree.unlink(node3,node2)
    OutTree.link(node1,node2) # recover the link between node1-node2
    print(scores)
    Format_OutTree(OutTree)
    #"""
    
n=4
AdjList='TCGGCCAA->4 \
4->TCGGCCAA \
CCTGGCTG->4 \
4->CCTGGCTG \
CACAGGAT->5 \
5->CACAGGAT \
TGAGTACC->5 \
5->TGAGTACC \
4->5 \
5->4'.split(' ')
T=NewTree.buildFromAdjList(AdjList,bidirectional=True) # here we consider undirected graph
SmallParsimony_Unrooted(T)

3 5 6
17.0
TCGGCCAA->CCTGGCAA:3
CCTGGCTG->CCTGGCAA:2
CACAGGAT->CATGGGAA:3
TGAGTACC->CATGGGAA:7
CCTGGCAA->TCGGCCAA:3
CCTGGCAA->CCTGGCTG:2
CCTGGCAA->CATGGGAA:2
CATGGGAA->CACAGGAT:3
CATGGGAA->CCTGGCAA:2
CATGGGAA->TGAGTACC:7


In [7]:
with open('ch8_data/dataset_369355_12.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    AdjList=lines[1::]
T=NewTree.buildFromAdjList(AdjList,bidirectional=True) # here we consider undirected graph
SmallParsimony_Unrooted(T)

61 56 62
524.0
AGTTCATACTGCCAGCCTTGGAGCGAATGT->AGATCATCATGCCAGGCTTTTAGCGTATGT:7
CGATGCCCATATCCGGTCCTTAGTCTATAC->AGATCATCATGCCAGGCTTTTAGCGTATGT:14
CGGGGTAGCCAGCCGCAACCGAGTCACGGT->GGACGTCGCCTGTCGGATCCATGTCTCAGT:12
GCACAACATTTGTCGGATTAATGCCTGAAA->GGACGTCGCCTGTCGGATCCATGTCTCAGT:12
ATAAGCAAGTGAAGCACCCTTTCCACGCGC->CTATGCCAGTTAATCACCCTTCTTATGCTC:10
CTTTCCCATTTCATTTAGATCCTTTTTCTC->CTATGCCAGTTAATCACCCTTCTTATGCTC:12
GAACGTATTATAAAGGATCGTCGTGACCTT->GAACGTCCCATATAGGATCCCTGTCTCTGT:11
TACGCACCCACTTAGCCGGCCTCGCTATGT->GAACGTCCCATATAGGATCCCTGTCTCTGT:14
CTGAGTTTTTGGGATGATTGGGCAGACTAT->CGGAGATTTTGAGAGGGTTCGACAGTCTGA:10
GGGGTACCCTAAGTGCGAGCTAAGCTGTGA->CGGAGATTTTGAGAGGGTTCGACAGTCTGA:16
CGCGAGTCACCAATGGATGGGATAACGATC->CGTGAGCCGCCAGTGGACGAGCTAACGATC:7
GGTACTCTGCAAGGACTCCAGCGAACGATG->CGTGAGCCGCCAGTGGACGAGCTAACGATC:13
GTTCGCGACGTGCGCAATACCCGCGTCCCT->GGACGTGGCGTGCGCGGTCCCCGCGTCTCT:8
TGATTTAGAGAGCGAGGACGTCTCTCATGA->GGACGTGGCGTGCGCGGTCCCCGCGTCTCT:16
TGACTTCAGGCTGCGACCCCACAGCGCGGG->TGATGCCAGCCAGTGACCGAGCATAACTGG:1

## 8.10 The Large Parsimony Problem

### Nearest Neighbors of a Tree Problem

In [27]:
from copy import deepcopy

def NearestNeighbors(AdjList,internalEdge):
    
    # internaledge (a,b)
    tmp=[]
    for edge in AdjList:
        if edge[0]==internalEdge[0] and edge!=internalEdge:
            tmp.append(edge)
    W,X=tmp # step.3 remaining nodes adjacent to a as W (a-w) and X (a-x)
    tmp=[]
    for edge in AdjList:
        if edge[0]==internalEdge[1] and edge!=internalEdge[::-1]:
            tmp.append(edge)
    Z,Y=tmp # step.3 remaining nodes adjacent to b as Y (b-y) and Z (b-z)
            
    print(W,X,Y,Z)

    newAdjList1=deepcopy(AdjList)
    newAdjList2=deepcopy(AdjList)

    # exchange X,Y (a-x),(b-y) -> (a-y),(b-x)
    newAdjList1.remove(X) # remove a->x
    newAdjList1.remove([X[1],X[0]]) # remove x->a
    newAdjList1.remove(Y) # remove b->y
    newAdjList1.remove([Y[1],Y[0]]) # remove y->b
    newAdjList1.append([X[0],Y[1]]) # add a->y
    newAdjList1.append([Y[1],X[0]]) # add y->a
    newAdjList1.append([Y[0],X[1]]) # add b->x
    newAdjList1.append([X[1],Y[0]]) # add x->b

    # exchange X,Z (a-x),(b,z) -> (a-z),(b,x)
    newAdjList2.remove(X)
    newAdjList2.remove([X[1],X[0]])
    newAdjList2.remove(Z)
    newAdjList2.remove([Z[1],Z[0]])
    newAdjList2.append([X[0],Z[1]]) # add a->z
    newAdjList2.append([Z[1],X[0]]) # add z->a
    newAdjList2.append([Z[0],X[1]]) # add b->x
    newAdjList2.append([X[1],Z[0]]) # add x->b
    
    return newAdjList1,newAdjList2

def FormatOutput(newAdjList1,newAdjList2):
    for x in newAdjList1:
        print('->'.join([str(y) for y in x]))
    print()
    for x in newAdjList2:
        print('->'.join([str(y) for y in x]))

internalEdge=[5,4]
AdjLink='0->4 \
4->0 \
1->4 \
4->1 \
2->5 \
5->2 \
3->5 \
5->3 \
4->5 \
5->4'.split(' ')
AdjList=[[int(x.split('->')[0]),int(x.split('->')[1])] for x in AdjLink]
newAdjList1,newAdjList2=NearestNeighbors(AdjList,internalEdge)
FormatOutput(newAdjList1,newAdjList2)

[5, 2] [5, 3] [4, 1] [4, 0]
0->4
4->0
2->5
5->2
4->5
5->4
5->1
1->5
4->3
3->4

1->4
4->1
2->5
5->2
4->5
5->4
5->0
0->5
4->3
3->4


In [47]:
def NearestNeighborsTree(Tree,internalEdge):
    
    tmp=[]
    for edge in Tree.edges[internalEdge[0]]:
        if edge[0]!=internal_edge[1]:
            tmp.append([internalEdge[0],edge[0]])
    W,X=tmp # step.3 remaining nodes adjacent to a as W (a-w) and X (a-x)
            
    tmp=[]
    for edge in Tree.edges[internalEdge[1]]:
        if edge[0]!=internal_edge[0]:
            tmp.append([internalEdge[1],edge[0]])
    Z,Y=tmp # step.3 remaining nodes adjacent to b as Y (b-y) and Z (b-z)

    print(W,X,Y,Z)

    newTree1=deepcopy(Tree)
    newTree2=deepcopy(Tree)

    # exchange X,Y (a-x),(b-y) -> (a-y),(b-x)
    newTree1.unlink(X[0],X[1])
    newTree1.unlink(X[1],X[0])
    newTree1.unlink(Y[0],Y[1])
    newTree1.unlink(Y[1],Y[0])
    newTree1.link(X[0],Y[1])
    newTree1.link(Y[1],X[0])
    newTree1.link(Y[0],X[1])
    newTree1.link(X[1],Y[0])

    # exchange X,Z (a-x),(b,z) -> (a-z),(b,x)
    newTree2.unlink(X[0],X[1])
    newTree2.unlink(X[1],X[0])
    newTree2.unlink(Z[0],Z[1])
    newTree2.unlink(Z[1],Z[0])
    newTree2.link(X[0],Z[1])
    newTree2.link(Z[1],X[0])
    newTree2.link(Z[0],X[1])
    newTree2.link(X[1],Z[0])

    return newTree1,newTree2

def FormatOutput(newTree1,newTree2):
    for x in newTree1.edges:
        for y in newTree1.edges[x]:
            print('%s->%s'%(str(x),str(y[0])))
    print()
    for x in newTree2.edges:
        for y in newTree2.edges[x]:
            print('%s->%s'%(str(x),str(y[0])))

internalEdge=[5,4]
AdjLink='0->4 \
4->0 \
1->4 \
4->1 \
2->5 \
5->2 \
3->5 \
5->3 \
4->5 \
5->4'.split(' ')
Tree=NewTree.buildFromAdjList(AdjLink) # here the AdjLink gives bidirectional, so no need to add bidirectional=True
newTree1,newTree2=NearestNeighborsTree(Tree,internalEdge)
FormatOutput(newTree1,newTree2)

[5, 2] [5, 3] [4, 1] [4, 0]
0->4
4->0
4->5
4->3
1->5
2->5
5->2
5->4
5->1
3->4

0->5
4->1
4->5
4->3
1->4
2->5
5->2
5->4
5->0
3->4


In [30]:
with open('ch8_data/dataset_369356_6.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    internalEdge=[int(y) for y in lines[0].split(' ')]
    AdjLink=lines[1::]
    AdjList=[[int(x.split('->')[0]),int(x.split('->')[1])] for x in AdjLink]
    newAdjList1,newAdjList2=NearestNeighbors(AdjList,internalEdge)
    FormatOutput(newAdjList1,newAdjList2)

[46, 28] [46, 29] [49, 57] [49, 41]
0->32
32->0
1->32
32->1
2->33
33->2
3->33
33->3
4->34
34->4
5->34
34->5
6->35
35->6
7->35
35->7
8->36
36->8
9->36
36->9
10->37
37->10
11->37
37->11
12->38
38->12
13->38
38->13
14->39
39->14
15->39
39->15
16->40
40->16
17->40
40->17
18->41
41->18
19->41
41->19
20->42
42->20
21->42
42->21
22->43
43->22
23->43
43->23
24->44
44->24
25->44
44->25
26->45
45->26
27->45
45->27
28->46
46->28
30->47
47->30
31->47
47->31
43->48
48->43
33->48
48->33
41->49
49->41
46->49
49->46
44->50
50->44
45->50
50->45
40->51
51->40
42->51
51->42
51->52
52->51
32->52
52->32
47->53
53->47
35->53
53->35
37->54
54->37
36->54
54->36
39->55
55->39
38->55
55->38
34->56
56->34
54->56
56->54
50->57
57->50
53->58
58->53
56->58
58->56
55->59
59->55
58->59
59->58
59->60
60->59
52->60
60->52
48->61
61->48
57->61
61->57
60->61
61->60
46->57
57->46
49->29
29->49

0->32
32->0
1->32
32->1
2->33
33->2
3->33
33->3
4->34
34->4
5->34
34->5
6->35
35->6
7->35
35->7
8->36
36->8
9->36
36->9
10->37
37

### NearestNeighborInterchange

In [35]:
import re,random
from copy import deepcopy

class Tree(object):
    
    ## public code for the data structure Tree
    
    def __init__(self,N=-1,bidirectional=True):
        self.nodes=list(range(N))
        self.edges={}
        self.bidirectional=bidirectional
        self.N = N
        
    def link(self,start,end,weight=1): 
        self.half_link(start,end,weight)
        if self.bidirectional:
            self.half_link(end,start,weight)
        
    def unlink(self,i,k):
        try:
            self.half_unlink(i,k)
            if self.bidirectional:
                self.half_unlink(k,i)
        except KeyError:
            print ('Could not unlink {0} from {1}'.format(i,k))
            self.print()        
        
    def half_link(self,a,b,weight=1):
        if not a in self.nodes:
            self.nodes.append(a)        
        if a in self.edges:               
            self.edges[a]=[(b0,w0) for (b0,w0) in self.edges[a] if b0 != b] + [(b,weight)]
        else:
            self.edges[a]=[(b,weight)]
    
    def half_unlink(self,a,b):
        links=[(e,w) for (e,w) in self.edges[a] if e != b]
        if len(links)<len(self.edges[a]):
            self.edges[a]=links
        else:
            print ('Could not unlink {0} from {1}'.format(a,b))
            self.print()
    
    def are_linked(self,a,b):
        return len([e for (e,w) in self.edges[a] if e == b])>0
        
    def AdjList(self,includeNodes=False):
        print('AdjList')
        self.nodes.sort()
        if includeNodes:
            print (self.nodes)
        for node in self.nodes:
            if node in self.edges:
                for edge in self.edges[node]:
                    end,weight=edge
                    print ('%i->%i:%.3f'%(node,end,weight))
                    
    def num_nodes(self):
        return len(self.nodes)
    
    def traverse(self,i,k,path=[],weights=[]):
        if not i in self.edges: return (False,[])
        if len(path)==0:
            path=[i]
            weights=[0]
        
        for j,w in self.edges[i]:
            if j in path: continue
            path1=path + [j]
            weights1=weights+[w]
            if j==k:
                return (True,list(zip(path1,weights1)))
            else:
                found_k,test=self.traverse(j,k,path1,weights1)
                if found_k: 
                    return (found_k,test)
        return (False,[])  
    
    def get_nodes(self):
        for node in self.nodes:
            yield(node)
     
    def get_links(self):
        return [(a,b,w) for a in self.nodes for (b,w) in self.edges[a] if a in self.edges]
    
    def remove_backward_links(self,root):
        self.bidirectional = False
        for (child,_) in self.edges[root]:
            self.half_unlink(child,root)
            self.remove_backward_links(child)
            
class NewTree(Tree):
        
    @staticmethod
    def buildFromAdjList(lines,letters='ATGC',bidirectional=False):
        def get_leaf(string):
            for index,label in T.labels.items():
                if string==label:
                    return index
            return None
        
        T=NewTree(bidirectional=bidirectional)
        pattern=re.compile('(([0-9]+)|([{0}]+))->(([0-9]+)|([{0}]+))'.format(letters))
        
        for line in lines:
            m=pattern.match(line)
            if m:
                i=None
 
                if m.group(2)==None:
                    i=get_leaf(m.group(3))
                    if i==None:
                        i=len(T.labels)
                    T.set_label(i,m.group(3))
 
                    if not i in T.leaves:
                        T.leaves.append(i)
                        
                if m.group(3)==None:
                    i=int(m.group(2))
           
                if m.group(5)==None:
                    j=get_leaf(m.group(6))
                    if j==None:
                        j=len(T.labels)
                        if not j in T.leaves:
                            T.leaves.append(j)                        
                        T.set_label(j,m.group(6))
                    T.link(i,j)
                   
                if m.group(6)==None:
                    j=int(m.group(5))
                    T.link(i,j)
                    
        return T
    
    def __init__(self,N=-1,bidirectional=True):
        super().__init__(N,bidirectional=bidirectional)
        self.labels={}
        self.leaves=[]
        
    def is_leaf(self,v):
        return v in self.leaves
    
    def set_label(self,node,label):
        if not node in self.nodes:
            self.nodes.append(node)
        self.labels[node]=label
            
def HamDistance(s,t):
    return len([a for (a,b) in zip(s,t) if a!=b])

import numpy as np

def findRipe(T,visited):
    for v in T.nodes:
        if not visited[v] and v in T.edges:
            for i,_ in T.edges[v]:
                if i>v:
                    continue
                if visited[i]==False:
                    break
            return v # all chlilds of v are visited, but v is not visited (Tag(v)=0)
    return None
        
def UpdateOutTree(v,scores,OutTree,alphabet='ATGC'):
    if v not in OutTree.labels:
        OutTree.labels[v]=''
    ii=0
    smallest_score=np.inf
    for i in range(len(scores)):
        if scores[i]<smallest_score:
            smallest_score=scores[i]
            ii=i
    OutTree.set_label(v,OutTree.labels[v]+alphabet[ii])
    return alphabet[ii]
        
def Backtrack(v,character_assign,scores,T,OutTree,alphabet='ATGC'):
    for v_,_ in T.edges[v]:
        if T.is_leaf(v_):
            continue
        if v_ not in OutTree.labels:
            OutTree.labels[v_]=''
        smallest_score=np.inf
        for i in range(len(alphabet)):
            if scores[v_][i]<smallest_score:
                smallest_score=scores[v_][i]
        moves=[i for i in range(len(alphabet)) if scores[v_][i]==smallest_score]
        flag=False
        for i in moves:
            if alphabet[i]==character_assign:        
                flag=True
                OutTree.set_label(v_,OutTree.labels[v_]+character_assign)
                Backtrack(v_,character_assign,scores,T,OutTree) 
        if flag==False:
            random_assign=alphabet[moves[random.randrange(0,(len(moves)))]]               
            OutTree.set_label(v_,OutTree.labels[v_]+random_assign)
            Backtrack(v_,random_assign,scores,T,OutTree)   
            
def CalScore(alp,v,scores,T,alphabet='ATGC'):
    tmp=[]
    for e,_ in T.edges[v]:
        ttmp=[]
        for i in range(len(alphabet)):
            if alp==alphabet[i]:
                delta=0
            else:
                delta=1
            ttmp.append(scores[e][i]+delta)
        tmp.append(np.min(ttmp))
    return sum(tmp)

def SmallParsimonyCharacter(Character,OutTree,T,alphabet='ATGC'):
            
    # new tree for solving single character smallest score
    visited={}
    scores={}
    print('Tree SmallParsimonyCharacter',T.nodes)
    for v in T.nodes:
        if T.is_leaf(v):
            visited[v]=True
            scores[v]=[]
            for alp in alphabet:
                if Character[v]==alp:
                    scores[v].append(0)
                else:
                    scores[v].append(np.inf)
        else:
            visited[v]=False
    print('scores SmallParsimonyCharacter',scores)
    v=findRipe(T,visited)
    print('v SmallParsimonyCharacter',v)
    while v!=None:
        visited[v]=True
        scores[v]=[CalScore(alp,v,scores,T) for alp in alphabet]
        _v=v # keep the last v
        v=findRipe(T,visited) # update all Ripe in T
    character_assign=UpdateOutTree(_v,scores[_v],OutTree)
    Backtrack(_v,character_assign,scores,T,OutTree)
    tmp=[]
    for i in range(len(alphabet)):
        tmp.append(scores[_v][i])
    smallest_score=min(tmp)
    return smallest_score
        
def SmallParsimony(T):
 
    # use recursion to get the minimal value
    OutTree=NewTree(T.N)
    print('Tree in SmallParsimony(T)',T.edges,T.labels,T.leaves)
    
    for node in T.nodes:
        if not node in OutTree.nodes:
            OutTree.nodes.append(node)
            if node in T.edges:
                for a,w in T.edges[node]:
                    OutTree.link(node,a,w)
    for node,label in T.labels.items():
        OutTree.set_label(node,label)
        
    tmp=[]
    #print(T.labels)
    for i in range(len(T.labels[0])): # for each position
        #print(i)
        #print([label[i] for label in T.labels.values()],OutTree.edges,OutTree.labels,T.edges,T.labels)
        tmp.append(SmallParsimonyCharacter([label[i] for label in T.labels.values()],OutTree,T))
    scores=sum(tmp)
    
    return scores,OutTree

def SmallParsimony_Unrooted(T):
    
    # in this case, we can assign a root
    #print(T.nodes,T.edges,T.labels)
    print('T SmallParsimony_Unrooted before',T.edges,T.leaves)
    node1=[x for x in T.nodes if x not in T.leaves][-1]
    node2=[x for x in T.edges[node1] if x not in T.leaves][0][0]
    #print(node1,node2)
    T.unlink(node1,node2)
    node3=len(T.nodes) # then node3 is the new root
    T.link(node3,node1)
    T.link(node3,node2)
    #print(node1,node2,node3)
    #"""
    T.remove_backward_links(node3)
    print('T SmallParsimony_Unrooted after',T.edges,T.leaves)
    scores,OutTree=SmallParsimony(T)
    OutTree.unlink(node3,node1)
    OutTree.unlink(node3,node2)
    OutTree.link(node1,node2) # recover the link between node1-node2
    
    ## important, remove the additional node
    OutTree.nodes.remove(node3)
    OutTree.edges.pop(node3)
    #print('Outtree',OutTree.edges)
    return scores,OutTree

def NearestNeighborsTree(Tree,internalEdge):
    
    tmp=[]
    for edge in Tree.edges[internalEdge[0]]:
        if edge[0]!=internalEdge[1]:
            #print(edge,internalEdge)
            tmp.append([internalEdge[0],edge[0]])
    #print(internalEdge,tmp,Tree.edges)
    W,X=tmp # step.3 remaining nodes adjacent to a as W (a-w) and X (a-x)
            
    tmp=[]
    for edge in Tree.edges[internalEdge[1]]:
        if edge[0]!=internalEdge[0]:
            tmp.append([internalEdge[1],edge[0]])
    Z,Y=tmp # step.3 remaining nodes adjacent to b as Y (b-y) and Z (b-z)

    print(W,X,Y,Z)

    newTree1=deepcopy(Tree)
    newTree2=deepcopy(Tree)

    # exchange X,Y (a-x),(b-y) -> (a-y),(b-x)
    newTree1.unlink(X[0],X[1])
    #newTree1.unlink(X[1],X[0])
    newTree1.unlink(Y[0],Y[1])
    #newTree1.unlink(Y[1],Y[0])
    newTree1.link(X[0],Y[1])
    newTree1.link(Y[1],X[0])
    newTree1.link(Y[0],X[1])
    newTree1.link(X[1],Y[0])

    # exchange X,Z (a-x),(b,z) -> (a-z),(b,x)
    newTree2.unlink(X[0],X[1])
    #newTree2.unlink(X[1],X[0])
    newTree2.unlink(Z[0],Z[1])
    #newTree2.unlink(Z[1],Z[0])
    newTree2.link(X[0],Z[1])
    newTree2.link(Z[1],X[0])
    newTree2.link(Z[0],X[1])
    newTree2.link(X[1],Z[0])

    return newTree1,newTree2

def NearestNeighborInterchange(AdjList):
    score=np.inf
    Tree=NewTree.buildFromAdjList(AdjList,bidirectional=True)
    print('Original Tree',Tree.edges,Tree.nodes,Tree.labels,Tree.bidirectional,Tree.N,Tree.leaves)
    newScore,newTree=SmallParsimony_Unrooted(Tree)
    Strings=[]
    for x in AdjList:
        try:
            int(x.split('->')[0])
        except:
            Strings.append(x.split('->')[0])
    while newScore<score:
        score=newScore
        Tree=newTree
        internal_edges=[]
        for v in Tree.nodes:
            for w in Tree.edges[v]:
                #print(v,w)
                if v not in list(range(len(Strings))) and w[0] not in list(range(len(Strings))):
                    internal_edges.append([v,w[0]])
        Tree.leaves=[x for x in range(len(Strings))]
        print('SP Tree',Tree.edges,Tree.leaves)
        print(internal_edges)
        for internal_edge in internal_edges:
            for NeighborTree in NearestNeighborsTree(Tree,internal_edge):
                print('Neighbor Tree',NeighborTree.edges,NeighborTree.nodes,NeighborTree.labels,NeighborTree.bidirectional,NeighborTree.N,NeighborTree.leaves)
                tmp=deepcopy(NeighborTree)
                for label_id in range(len(Strings),len(Tree.nodes)+1):
                    tmp.labels.pop(label_id)
                print('tmp Tree',tmp.edges,tmp.nodes,tmp.labels,tmp.bidirectional,tmp.N,tmp.leaves)
                _,neighborScore=SmallParsimony_Unrooted(tmp)
                if neighborScore<newScore:
                    newScore=neighborScore
                    newTree=tmp
    return newTree
    
AdjList='GCAGGGTA->5 \
TTTACGCG->5 \
CGACCTGA->6 \
GATTCCAC->6 \
5->TTTACGCG \
5->GCAGGGTA \
5->7 \
TCCGTAGT->7 \
7->5 \
7->6 \
7->TCCGTAGT \
6->GATTCCAC \
6->CGACCTGA \
6->7'.split(' ')
NearestNeighborInterchange(AdjList)

Original Tree {0: [(5, 1)], 5: [(1, 1), (0, 1), (7, 1)], 1: [(5, 1)], 2: [(6, 1)], 6: [(3, 1), (2, 1), (7, 1)], 3: [(6, 1)], 7: [(5, 1), (4, 1), (6, 1)], 4: [(7, 1)]} [0, 5, 1, 2, 6, 3, 7, 4] {0: 'GCAGGGTA', 1: 'TTTACGCG', 2: 'CGACCTGA', 3: 'GATTCCAC', 4: 'TCCGTAGT'} True -1 [0, 1, 2, 3, 4]
T SmallParsimony_Unrooted before {0: [(5, 1)], 5: [(1, 1), (0, 1), (7, 1)], 1: [(5, 1)], 2: [(6, 1)], 6: [(3, 1), (2, 1), (7, 1)], 3: [(6, 1)], 7: [(5, 1), (4, 1), (6, 1)], 4: [(7, 1)]} [0, 1, 2, 3, 4]
T SmallParsimony_Unrooted after {0: [], 5: [(1, 1), (0, 1)], 1: [], 2: [], 6: [(3, 1), (2, 1)], 3: [], 7: [(4, 1), (6, 1)], 4: [], 8: [(7, 1), (5, 1)]} [0, 1, 2, 3, 4]
Tree in SmallParsimony(T) {0: [], 5: [(1, 1), (0, 1)], 1: [], 2: [], 6: [(3, 1), (2, 1)], 3: [], 7: [(4, 1), (6, 1)], 4: [], 8: [(7, 1), (5, 1)]} {0: 'GCAGGGTA', 1: 'TTTACGCG', 2: 'CGACCTGA', 3: 'GATTCCAC', 4: 'TCCGTAGT'} [0, 1, 2, 3, 4]
Tree SmallParsimonyCharacter [0, 5, 1, 2, 6, 3, 7, 4, 8]
scores SmallParsimonyCharacter {0: [inf, in

KeyError: 6

In [38]:
with open('ch8_data/dataset_369356_8.txt','r') as f:
    lines=[x.rstrip('\n') for x in f.readlines()]
    n=int(lines[0])
    AdjList=lines[1::]
    NearestNeighborInterchange([x.split('->') for x in AdjList])

158
TCCATCACTCGGGACGGTGCACCAAGCACGTTATCACGAT->TTTAGGGCGTGGTTCTTGGAACCCAGCACGTCACGAAGAG:19
TTTAGGGCGTGGTTCTTGGAACCCAGCACGTCACGAAGAG->TCCATCACTCGGGACGGTGCACCAAGCACGTTATCACGAT:19
TCCATCACTCGGGACGGTGCACCAAGCACGTTATCACGAT->TCCCTCACTCAGGACGGTGCAAGAGAGGTATTCTCTCAAT:13
TCCCTCACTCAGGACGGTGCAAGAGAGGTATTCTCTCAAT->TCCATCACTCGGGACGGTGCACCAAGCACGTTATCACGAT:13
TCGATACCTCCCGCCAGCCCCCCAGGAAAGTTGTCGCGAT->GCGCTACCTTCCGCTAGCCACCTAGGGAAGCGGATGGTAT:13
GCGCTACCTTCCGCTAGCCACCTAGGGAAGCGGATGGTAT->TCGATACCTCCCGCCAGCCCCCCAGGAAAGTTGTCGCGAT:13
TCGATACCTCCCGCCAGCCCCCCAGGAAAGTTGTCGCGAT->TAAAAGCATCCCTGCATCGCTGCAGAATGATTGTCCCGAA:17
TAAAAGCATCCCTGCATCGCTGCAGAATGATTGTCCCGAA->TCGATACCTCCCGCCAGCCCCCCAGGAAAGTTGTCGCGAT:17
TGAGAACACGGCACCGCTACAGGCAACCATCTCTTCCGAT->TGAGGAGTCGCCTGACCCACGGGGACCCATCTCTTTCGAT:13
TGAGGAGTCGCCTGACCCACGGGGACCCATCTCTTTCGAT->TGAGAACACGGCACCGCTACAGGCAACCATCTCTTCCGAT:13
TGAGAACACGGCACCGCTACAGGCAACCATCTCTTCCGAT->GGATACCATTGAACCGTTCCAATCAATTACCACTTCGAGT:17
GGATACCATTGAACCGTTCCAATCAATTACCACTTCGAGT->TGAGAACA

151
TCGATGGCCAGGTCCCTTGCACCCAGCACGTTGCCGAGAC->TTTAGGGCGTGGTTCTTGGAACCCAGCACGTCACGAAGAG:14
TTTAGGGCGTGGTTCTTGGAACCCAGCACGTCACGAAGAG->TCGATGGCCAGGTCCCTTGCACCCAGCACGTTGCCGAGAC:14
TCGCTACCTCCCGCCAGTGCACGAGAGAAATTGTCGCAAT->GCGCTACCTTCCGCTAGCCACCTAGGGAAGCGGATGGTAT:16
GCGCTACCTTCCGCTAGCCACCTAGGGAAGCGGATGGTAT->TCGCTACCTCCCGCCAGTGCACGAGAGAAATTGTCGCAAT:16
TGAGAACATGCCTGCGTTACAGGCAACTATCTCTTCCGAT->TGAGGAGTCGCCTGACCCACGGGGACCCATCTCTTTCGAT:13
TGAGGAGTCGCCTGACCCACGGGGACCCATCTCTTTCGAT->TGAGAACATGCCTGCGTTACAGGCAACTATCTCTTCCGAT:13
TGAGAACATGCCTGCGTTACAGGCAACTATCTCTTCCGAT->GGATACCATTGAACCGTTCCAATCAATTACCACTTCGAGT:17
GGATACCATTGAACCGTTCCAATCAATTACCACTTCGAGT->TGAGAACATGCCTGCGTTACAGGCAACTATCTCTTCCGAT:17
TAAGAACATGCCTGCGTTACAGACAACTATTTCTTCCGAA->AAGGAATCGGTCGAGGGTATCGACAGCGTTTTTTTGCGTA:18
AAGGAATCGGTCGAGGGTATCGACAGCGTTTTTTTGCGTA->TAAGAACATGCCTGCGTTACAGACAACTATTTCTTCCGAA:18
TAAAAGCATCCCTGCATTGCAGCCAACTAATTGTCCCGAA->TAAGAACATGCCTGCGTTACAGACAACTATTTCTTCCGAA:9
TAAGAACATGCCTGCGTTACAGACAACTATTTCTTCCGAA->TAAAAGCAT

In [36]:
## public code cited online

import sys
from copy import deepcopy

ALPHABET = ['A', 'C', 'G', 'T']

def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def process_input(AdjList):
    for edge in AdjList:
        if RepresentsInt(edge[0]) and RepresentsInt(edge[1]):
            break

    AdjList.remove([edge[0], edge[1]])
    AdjList.remove([edge[1], edge[0]])

    AdjList += [[1e6, edge[0]], [1e6, edge[1]]]

    visited = [False] * len(AdjList)
    # visited = visited[:-2] + [True, True]

    node_list = [1e6]
    to_remove = []
    while len(node_list) != 0:
        current_node = node_list[-1]
        possible_edges = [e for i, e in enumerate(AdjList) if e[0] == current_node and visited[i] == False]
        if len(possible_edges) != 0:
            current_edge = possible_edges[0]
            node_list.append(current_edge[1])
            ## remove reciprocal
            tmp = [current_edge[1], current_edge[0]]
            to_remove.append(tmp)
            for i in range(len(visited)):
                if AdjList[i] == tmp:
                    visited[i] = True
                elif AdjList[i] == current_edge:
                    visited[i] = True
        else:
            del node_list[-1]

    AdjList = [x for x in AdjList if x not in to_remove]

    return AdjList

def HammingDistance(p, q):
    mm = [p[i] != q[i] for i in range(len(p))]
    return sum(mm)

def SmallParsimony(AdjList):
    ## initialize
    Tag = {}
    S = {}

    nodes = [item for sublist in AdjList for item in sublist]
    nodes = list(set(nodes))

    for v in nodes:
        S[v] = {}
        Tag[v] = 0
        if not RepresentsInt(v):
            Tag[v] = 1
            len_dna = len(v)
            for pos in range(len_dna):
                S[v][pos] = {}
                char = v[pos]
                for k in ALPHABET:
                    if char == k:
                        S[v][pos][k] = 0
                    else:
                        S[v][pos][k] = 1e6

    ## calculate scores
    while any(x == 0 for x in list(Tag.values())):
        zero_nodes = [node for node, tag in Tag.items() if tag == 0]
        for zn in zero_nodes:
            children = [child for parent, child in AdjList if parent == zn]
            if all([Tag[child] == 1 for child in children]):
                v = zn
                break
        Tag[v] = 1
        S[v] = {}
        for pos in range(len_dna):
            S[v][pos] = {}
            for k in ALPHABET:
                temp = []
                for i, score in S[children[0]][pos].items():
                    if i == k:
                        temp.append(score)
                    else:
                        temp.append(score + 1)
                score_daughter = min(temp)

                temp = []
                for i, score in S[children[1]][pos].items():
                    if i == k:
                        temp.append(score)
                    else:
                        temp.append(score + 1)
                score_son = min(temp)
                
                S[v][pos][k] = score_daughter + score_son
    return S

def FinalTree(AdjList, score_dict):
    nodes = [item for sublist in AdjList for item in sublist]
    nodes = list(set(nodes))
    child_nodes = [child for parent, child in AdjList]

    ## Find root
    root = nodes[0]
    idx = 1
    while root in child_nodes:
        root = nodes[idx]
        idx += 1

    ## Root's label and min parsimony score
    label_dict = {}
    label_dict[root] = ''
    min_pars_score = 0
    for pos, scores in score_dict[root].items():
        label_dict[root] += min(scores, key = scores.get)
        min_pars_score += min(scores.values())

    ## Backtrace
    Tag = {}
    for node in nodes:
        if not RepresentsInt(node):
            Tag[node] = 1
        else:
            Tag[node] = 0

    Tag[root] = 1

    one_nodes = [node for node, tag in Tag.items() if tag == 1 and RepresentsInt(node)]
    while any(x == 0 for x in list(Tag.values())):

        for node in one_nodes:
            children = [child for parent, child in AdjList if parent == node]
            if any([Tag[child] == 0 for child in children]):
                v = node
                break
        one_nodes += [child for child in children if RepresentsInt(child)]
        one_nodes = [x for x in one_nodes if x != v]

        if RepresentsInt(children[0]):
            daughter_label = ''
            daughter_scores = score_dict[children[0]]
            for pos, daughter_score in daughter_scores.items():
                parent_letter = label_dict[v][pos]
                min_nucs = [nuc for nuc, val in daughter_score.items() if val == min(daughter_score.values())]
                if parent_letter in min_nucs:
                    daughter_label += parent_letter
                else:
                    daughter_label += min_nucs[0]

            label_dict[children[0]] = daughter_label
            Tag[children[0]] = 1

        if RepresentsInt(children[1]):
            son_label = ''
            son_scores = score_dict[children[1]]
            for pos, son_score in son_scores.items():
                parent_letter = label_dict[v][pos]
                min_nucs = [nuc for nuc, val in son_score.items() if val == min(son_score.values())]
                if parent_letter in min_nucs:
                    son_label += parent_letter
                else:
                    son_label += min_nucs[0]

            label_dict[children[1]] = son_label
            Tag[children[1]] = 1

    ## Create final adjacency list
    final_AdjList = []
    for edge in AdjList:
        if RepresentsInt(edge[0]):
            node0 = label_dict[edge[0]]
        else:
            node0 = edge[0]
        if RepresentsInt(edge[1]):
            node1 = label_dict[edge[1]]
        else:
            node1 = edge[1]
        final_AdjList.append([node0, node1, HammingDistance(node0, node1)])
        final_AdjList.append([node1, node0, HammingDistance(node0, node1)])

    return [final_AdjList, min_pars_score]

def SmallParsUnrooted_wrapper(AdjList):
    AdjList = process_input(AdjList)
    score_dict = SmallParsimony(AdjList)


    final_AdjList, min_pars_score = FinalTree(AdjList, score_dict)

    tmp = final_AdjList[-4:]
    to_add1 = tmp[0][1]
    to_add2 = tmp[2][1]
    final_AdjList = final_AdjList[:-4]
    final_AdjList.append([to_add1, to_add2, HammingDistance(to_add1, to_add2)])
    final_AdjList.append([to_add2, to_add1, HammingDistance(to_add1, to_add2)])

    return [final_AdjList, min_pars_score]

def NearestNeighbors(AdjList, chosen_edge):

    w, x = [edge for edge in AdjList if edge[0] == chosen_edge[0] and edge != chosen_edge]
    y, z = [edge for edge in AdjList if edge[0] == chosen_edge[1] and edge != chosen_edge[::-1]]

    new_adj1 = deepcopy(AdjList)
    new_adj2 = deepcopy(AdjList)

    ## create nearest neighbour 1
    new_adj1.remove(x)
    new_adj1.remove(x[::-1])
    new_adj1.remove(y)
    new_adj1.remove(y[::-1])

    new_adj1 += [[y[0], x[1]], [x[0], y[1]], [x[1], y[0]], [y[1], x[0]]]

    ## create nearest neighbour 2
    new_adj2.remove(x)
    new_adj2.remove(x[::-1])
    new_adj2.remove(z)
    new_adj2.remove(z[::-1])

    new_adj2 += [[z[0], x[1]], [x[0], z[1]], [x[1], z[0]], [z[1], x[0]]]

    return [new_adj1, new_adj2]

def printTree(min_pars_score, final_AdjList):
    print(min_pars_score)
    for edge in final_AdjList:
        print(str(edge[0]) + '->' + str(edge[1]) + ':' +  str(edge[2]))
    print()
    return None

def NearestNeighborInterchange(AdjList):

    org_list = deepcopy(AdjList)
    strings = [x[0] for x in AdjList if not RepresentsInt(x[0])]
    score = 1e6
    newTree, newScore = SmallParsUnrooted_wrapper(AdjList)
    newTree = org_list
    while newScore < score:
        score = newScore
        tree = newTree

        if tree != org_list:
            tmp = deepcopy(tree)
            currentTree, currentScore = SmallParsUnrooted_wrapper(tmp)
            printTree(currentScore, currentTree)
        
        for edge in [e for e in tree if e[0] not in strings and e[1] not in strings]:
            NeighborTrees = NearestNeighbors(tree, edge)
            for NTree in NeighborTrees:
                tmp = deepcopy(NTree)
                x, neighborScore = SmallParsUnrooted_wrapper(tmp)
                if neighborScore < newScore:
                    newScore = neighborScore
                    newTree = NTree

n=5
AdjList='GCAGGGTA->5 \
TTTACGCG->5 \
CGACCTGA->6 \
GATTCCAC->6 \
5->TTTACGCG \
5->GCAGGGTA \
5->7 \
TCCGTAGT->7 \
7->5 \
7->6 \
7->TCCGTAGT \
6->GATTCCAC \
6->CGACCTGA \
6->7'.split(' ')
NearestNeighborInterchange([x.split('->') for x in AdjList])


22
TCCGCGGG->TTTACGCG:4
TTTACGCG->TCCGCGGG:4
GCAGCGGA->GAACCCGA:3
GAACCCGA->GCAGCGGA:3
GAACCCGA->GATTCCAC:4
GATTCCAC->GAACCCGA:4
GAACCCGA->CGACCTGA:3
CGACCTGA->GAACCCGA:3
GCAGCGGA->GCAGGGTA:2
GCAGGGTA->GCAGCGGA:2
TCCGCGGG->TCCGTAGT:3
TCCGTAGT->TCCGCGGG:3
TCCGCGGG->GCAGCGGA:3
GCAGCGGA->TCCGCGGG:3

21
TCTGCGGG->TTTACGCG:3
TTTACGCG->TCTGCGGG:3
GCTGCGGA->GCAGCGGA:1
GCAGCGGA->GCTGCGGA:1
GCAGCGGA->CGACCTGA:4
CGACCTGA->GCAGCGGA:4
TCTGCGGG->TCCGTAGT:4
TCCGTAGT->TCTGCGGG:4
GCAGCGGA->GCAGGGTA:2
GCAGGGTA->GCAGCGGA:2
GCTGCGGA->GATTCCAC:5
GATTCCAC->GCTGCGGA:5
TCTGCGGG->GCTGCGGA:2
GCTGCGGA->TCTGCGGG:2

